# 1. Load packages

The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

or the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

In [1]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd## 1.1 Define output folder

output_main_folder = r'L:\Shiwei\Figures\MOp_draft_2023_v2\Nuclear_organization_v2'
output_analysis_folder = os.path.join(output_main_folder, 'analysis')
output_figure_folder = os.path.join(output_main_folder, 'figures')

make_output_folder = True

if make_output_folder and not os.path.exists(output_analysis_folder):
    os.makedirs(output_analysis_folder)
    print(f'Generating analysis folder: {output_analysis_folder}.')
elif os.path.exists(output_analysis_folder):
    print(f'Use existing analysis folder: {output_analysis_folder}.')
    
if make_output_folder and not os.path.exists(output_figure_folder):
    os.makedirs(output_figure_folder)
    print(f'Generating figure folder: {output_figure_folder}.')
elif os.path.exists(output_figure_folder):
    print(f'Use existing figure folder: {output_figure_folder}.')

36556
Use existing analysis folder: L:\Shiwei\Figures\MOp_draft_2023_v2\Nuclear_organization_v2\analysis.
Use existing figure folder: L:\Shiwei\Figures\MOp_draft_2023_v2\Nuclear_organization_v2\figures.


In [2]:
import seaborn as sns
import scanpy as sc
import numpy as np

## 1.1 Define output folder

In [3]:
output_main_folder = r'L:\Shiwei\Figures\MOp_draft_2023_v2\Supplementary_tables'
output_sub_folder = os.path.join(output_main_folder, 'Readout_and_Primer')
#output_figure_folder = os.path.join(output_main_folder, 'figures')

make_output_folder = True

if output_main_folder and not os.path.exists(output_main_folder):
    os.makedirs(output_main_folder)
    print(f'Generating output folder: {output_main_folder}.')
elif os.path.exists(output_analysis_folder):
    print(f'Use existing output folder: {output_main_folder}.')

    
if output_main_folder and not os.path.exists(output_sub_folder):
    os.makedirs(output_sub_folder)
    print(f'Generating output subfolder: {output_sub_folder}.')
elif os.path.exists(output_sub_folder):
    print(f'Use existing output subfolder: {output_sub_folder}.')

Use existing output folder: L:\Shiwei\Figures\MOp_draft_2023_v2\Supplementary_tables.
Generating output subfolder: L:\Shiwei\Figures\MOp_draft_2023_v2\Supplementary_tables\Readout_and_Primer.


In [4]:
library_main_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries'

# 2. Convert primers for each library

In [69]:
T7_seq = r'TAATACGACTCACTATAG'

### Load CTP summary

In [20]:
summary_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Probe_summary_from_PZ'
summary_fname = os.path.join(summary_folder, 'Chromatin_Libraries.xlsx')

summary_df = pd.read_excel(summary_fname)
summary_df = summary_df.fillna(method='ffill')

sel_Sublibrary_list = ['1000-mouse-genome_500', 
                       '1000-mouse-genome_250_1',
                       '1000-mouse-genome_250_2',
                       'marker_gene_TSS',
                       'mouse_brain_SE+sequential']

summary_df = summary_df[summary_df['Sublibrary'].isin(sel_Sublibrary_list)]
summary_df = summary_df[summary_df.columns[:6]]
summary_df = summary_df.drop_duplicates()
summary_df

,Library,Sublibrary,Forward primer name,Reverse primer name,Forward primer seq,Reverse primer seq
34,CTP11_mouse-genome,1000-mouse-genome_500,W1A03_primer_2,W1A10_primer_9,CCCGCAATGGCTGACAACCG,TAATACGACTCACTATAGGGATTGCCGCATGGTTTCCG
35,CTP11_mouse-genome,1000-mouse-genome_250_1,W1A07_primer_6,W1A12_primer_11,CGCAAACTGGTGCGGAAGGC,TAATACGACTCACTATAGGGCCATTGCCCGCGAGGTCGAG
36,CTP11_mouse-genome,1000-mouse-genome_250_2,W1B05_primer_16,W1B04_primer_15,CGGGTTTCGTTGCGCACACC,TAATACGACTCACTATAGGGCTTGTGCATCGCGCCAAAGA
41,CTP12_mouse_brain_gene,marker_gene_TSS,W1A07_primer_6,W1A12_primer_11,CGCAAACTGGTGCGGAAGGC,TAATACGACTCACTATAGGGCCATTGCCCGCGAGGTCGAG
44,CTP13_mouse_SE,mouse_brain_SE+sequential,W1A03_primer_2,W1A10_primer_9,CCCGCAATGGCTGACAACCG,TAATACGACTCACTATAGGGATTGCCGCATGGTTTCCG


In [21]:
descrip_rename_dict = {'CTP11_mouse-genome':'Genome chromatin', 
                       'CTP12_mouse_brain_gene':'Marker chromatin', 
                       'CTP13_mouse_SE':'Super-enhancer chromatin', 
                       '1000-mouse-genome_500':'Genome chromatin_set_1_and_2', 
                       '1000-mouse-genome_250_1':'Genome chromatin_set_3',
                       '1000-mouse-genome_250_2':'Genome chromatin_set_4', 
                       'marker_gene_TSS':'Marker chromatin_set_all', 
                       'mouse_brain_SE+sequential':'Super-enhancer chromatin_set_all',}

summary_df['Library']= summary_df['Library'].apply(lambda x: descrip_rename_dict[x])
summary_df['Sublibrary']= summary_df['Sublibrary'].apply(lambda x: descrip_rename_dict[x])
summary_df

,Library,Sublibrary,Forward primer name,Reverse primer name,Forward primer seq,Reverse primer seq
34,Genome chromatin,Genome chromatin_set_1_and_2,W1A03_primer_2,W1A10_primer_9,CCCGCAATGGCTGACAACCG,TAATACGACTCACTATAGGGATTGCCGCATGGTTTCCG
35,Genome chromatin,Genome chromatin_set_3,W1A07_primer_6,W1A12_primer_11,CGCAAACTGGTGCGGAAGGC,TAATACGACTCACTATAGGGCCATTGCCCGCGAGGTCGAG
36,Genome chromatin,Genome chromatin_set_4,W1B05_primer_16,W1B04_primer_15,CGGGTTTCGTTGCGCACACC,TAATACGACTCACTATAGGGCTTGTGCATCGCGCCAAAGA
41,Marker chromatin,Marker chromatin_set_all,W1A07_primer_6,W1A12_primer_11,CGCAAACTGGTGCGGAAGGC,TAATACGACTCACTATAGGGCCATTGCCCGCGAGGTCGAG
44,Super-enhancer chromatin,Super-enhancer chromatin_set_all,W1A03_primer_2,W1A10_primer_9,CCCGCAATGGCTGACAACCG,TAATACGACTCACTATAGGGATTGCCGCATGGTTTCCG


In [52]:
len('CCCGCAATGGCTGACAACCG')

20

### Inspect CTP oligprobes

In [48]:
primary_folder = os.path.join(output_main_folder,'Primary')
primary_fname = os.path.join(primary_folder, 'Super-enhancer chromatin probes.csv')

primary_df = pd.read_csv(primary_fname)
primary_df['sequence'][:3].tolist()

['CCCGCAATGGCTGACAACCGTAGATAGAGACGTTCGTGAACTCCCTATTGACCTCACTCTATGGTACATCCTCTGTTATGACGATTTATCTATGACCGGGATGACAGGACGTACCGCTTACCCGGAAACCATGCGGCAATCC',
 'CCCGCAATGGCTGACAACCGTAGATAGAGACGTTCGTGAAAAAGCTCTGGAATTCTTTTGTTATGAGCCTCATTAGTGCTGGGATTTATCTATGACCGGGATGACAGGACGTACCGCTTACCCGGAAACCATGCGGCAATCC',
 'CCCGCAATGGCTGACAACCGTAGATAGAGACGTTCGTGAACACAGAAAAGGGCAACGAGGGCAATGCGGAGATCATTTACTCGATTTATCTATGACCGGGATGACAGGACGTACCGCTTACCCGGAAACCATGCGGCAATCC']

In [54]:
print(len(primary_df['sequence'][:3].tolist()[0]))

primary_df['sequence'][:3].tolist()[0][-20:]

142


'CGGAAACCATGCGGCAATCC'

In [56]:
reverse_T7 = summary_df['Reverse primer seq'].tolist()[-1]
print(reverse_T7)
reverse_wo_T7 = reverse_T7[len(T7_seq):]
print(reverse_wo_T7)

from Bio import SeqIO
from Bio.Seq import Seq


Seq(reverse_wo_T7).reverse_complement()

TAATACGACTCACTATAGGGATTGCCGCATGGTTTCCG
GGATTGCCGCATGGTTTCCG


Seq('CGGAAACCATGCGGCAATCC')

### Extract from RNA merfish oligoprobes

In [59]:
primary_folder = os.path.join(output_main_folder,'Primary')
primary_fname = os.path.join(primary_folder, 'RNA MERFISH probes.csv')

In [60]:
primary_df = pd.read_csv(primary_fname)
primary_df = primary_df[~primary_df['name'].str.contains('_sequential')]
primary_df['sequence'][:3].tolist()

['CGCGGTGGTGGTACGACATGAAGAGTGAGTAGTAGTGGAGTGTTAAGAACTGGGGAGCTGGAAAATGAGTGAAGGGTGTGTTTGTAAAGGGTAGCGCAACTAGGACGGACAA',
 'CGCGGTGGTGGTACGACATGAAGGAGGAGGGTAATGATAGAGGAGGCTTCTCTTTCTAGTACATTTCTGCAAAGAGTGAGTAGTAGTGGAGTAGCGCAACTAGGACGGACAA',
 'CGCGGTGGTGGTACGACATGAAGAGTGAGTAGTAGTGGAGTTTTTGGAGTTCAGGCTGTGGGTACAGACCTATGGGAGGATTGAGATGAGTTAGCGCAACTAGGACGGACAA']

In [66]:
print(primary_df['sequence'][:3].tolist()[0][:20])
forward_seq = primary_df['sequence'][:3].tolist()[0][:20]

# check
print(np.sum(np.array([n[:20] for n in primary_df['sequence'].tolist()])==forward_seq))
print(len(primary_df))

CGCGGTGGTGGTACGACATG
22005
22005


In [68]:
reverse_seq_rc = primary_df['sequence'][:3].tolist()[0][-20:]
print(reverse_seq_rc)

# check
print(np.sum(np.array([n[-20:] for n in primary_df['sequence'].tolist()])==reverse_seq_rc))
print(len(primary_df))

AGCGCAACTAGGACGGACAA
22005
22005


In [73]:
from Bio import SeqIO
from Bio.Seq import Seq

reverse_wo_T7 = str(Seq(reverse_seq_rc).reverse_complement())
print(reverse_wo_T7)

print(Seq(reverse_wo_T7).reverse_complement() == primary_df['sequence'][:3].tolist()[0][-20:])

reverse_T7 = T7_seq + reverse_wo_T7
print(reverse_T7)

TTGTCCGTCCTAGTTGCGCT
True
TAATACGACTCACTATAGTTGTCCGTCCTAGTTGCGCT


In [83]:
merfish_primer_dict = {'Library': 'RNA MERFISH', 
                       'Sublibrary': 'RNA MERFISH_set_all', 
                       'Forward primer name': 'MOp_primer_F',
                       'Reverse primer name': 'MOp_primer_R',
                       'Forward primer seq': forward_seq,
                       'Reverse primer seq': reverse_T7,}

merfish_primer_df = pd.DataFrame(merfish_primer_dict, index=[0])
merfish_primer_df

,Library,Sublibrary,Forward primer name,Reverse primer name,Forward primer seq,Reverse primer seq
0,RNA MERFISH,RNA MERFISH_set_all,MOp_primer_F,MOp_primer_R,CGCGGTGGTGGTACGACATG,TAATACGACTCACTATAGTTGTCCGTCCTAGTTGCGCT


## Merge all primers

In [84]:
merged_summary_df = pd.concat([summary_df, merfish_primer_df])
merged_summary_df

,Library,Sublibrary,Forward primer name,Reverse primer name,Forward primer seq,Reverse primer seq
34,Genome chromatin,Genome chromatin_set_1_and_2,W1A03_primer_2,W1A10_primer_9,CCCGCAATGGCTGACAACCG,TAATACGACTCACTATAGGGATTGCCGCATGGTTTCCG
35,Genome chromatin,Genome chromatin_set_3,W1A07_primer_6,W1A12_primer_11,CGCAAACTGGTGCGGAAGGC,TAATACGACTCACTATAGGGCCATTGCCCGCGAGGTCGAG
36,Genome chromatin,Genome chromatin_set_4,W1B05_primer_16,W1B04_primer_15,CGGGTTTCGTTGCGCACACC,TAATACGACTCACTATAGGGCTTGTGCATCGCGCCAAAGA
41,Marker chromatin,Marker chromatin_set_all,W1A07_primer_6,W1A12_primer_11,CGCAAACTGGTGCGGAAGGC,TAATACGACTCACTATAGGGCCATTGCCCGCGAGGTCGAG
44,Super-enhancer chromatin,Super-enhancer chromatin_set_all,W1A03_primer_2,W1A10_primer_9,CCCGCAATGGCTGACAACCG,TAATACGACTCACTATAGGGATTGCCGCATGGTTTCCG
0,RNA MERFISH,RNA MERFISH_set_all,MOp_primer_F,MOp_primer_R,CGCGGTGGTGGTACGACATG,TAATACGACTCACTATAGTTGTCCGTCCTAGTTGCGCT


In [85]:
_name = 'Primers'
csv_fname = os.path.join(output_sub_folder, f'{_name}.csv')
if not os.path.exists(csv_fname):
    print(f'Save {_name}')
    merged_summary_df.to_csv(csv_fname, index=False)
else:
    print(f'Skip {_name}')

Save Primers


# 3. Convert common readouts

## Manual input using order quote

In [88]:
# \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Readouts\common_readouts
# QuoteRequest-QTE-163948.pdf

readout_dict = {'name': ['Stv82_ss_Alex750N', 
                         'Stv1_ss_Alex647N', 
                         'Stv79_ss_Atto565N'], 
                
                
                'sequence': ['/5Alex750N//iThioMC6-D/G TCA AGG ACG GCA GTG CAA A',
                             '/5Alex647N//iThioMC6-D/G GGA CGG TTC CAA TCG GAT C',
                             '/5ATTO565N//iThioMC6-D/T GAA AGC CGG ACA GTT CGC A']}

readout_df = pd.DataFrame(readout_dict)
readout_df

,name,sequence
0,Stv82_ss_Alex750N,/5Alex750N//iThioMC6-D/G TCA AGG ACG GCA GTG C...
1,Stv1_ss_Alex647N,/5Alex647N//iThioMC6-D/G GGA CGG TTC CAA TCG G...
2,Stv79_ss_Atto565N,/5ATTO565N//iThioMC6-D/T GAA AGC CGG ACA GTT C...


## Inspect by adaptors

In [95]:
adaptor_folder = os.path.join(output_main_folder,'Adaptor')
adaptor_fname = os.path.join(adaptor_folder, 'Marker chromatin adaptors.csv')

adaptor_df = pd.read_csv(adaptor_fname)
print(adaptor_df[:3]['sequence'].tolist())

adaptor_df[:3]

['TGACGTCGGTTTGGACGAGATTTGCACTGCCGTCCTTGACTTTGCACTGCCGTCCTTGAC', 'CGTTCGTACCGCGTACTTCGGATCCGATTGGAACCGTCCCGATCCGATTGGAACCGTCCC', 'CCCTTGTGAGCGCCCGACATTGCGAACTGTCCGGCTTTCATGCGAACTGTCCGGCTTTCA']


,name,sequence
0,Bit-1-Stv_91_2xStv_82rc,TGACGTCGGTTTGGACGAGATTTGCACTGCCGTCCTTGACTTTGCA...
1,Bit-2-Stv_32_2xStv_1rc,CGTTCGTACCGCGTACTTCGGATCCGATTGGAACCGTCCCGATCCG...
2,Bit-3-Stv_92_2xStv_79rc,CCCTTGTGAGCGCCCGACATTGCGAACTGTCCGGCTTTCATGCGAA...


In [124]:
from Bio import SeqIO
from Bio.Seq import Seq

for sel_idx in [0,1,2]:
    
    _ch = readout_df['name'].tolist()[sel_idx]
    print(f'Check {_ch}')
    
    sel_adaptor = adaptor_df[:3]['sequence'].tolist()[sel_idx]
    #print(len(sel_adaptor))
    #print(sel_adaptor)

    readout_from_adaptor = adaptor_df[:3]['sequence'].tolist()[0][-20:]
    #print(len(readout_from_adaptor))
    print('readout on adaptor:', readout_from_adaptor)
    print('  ')
    #readout_seq = str(Seq(readout_from_adaptor).reverse_complement())
    #readout_seq

    readout_seq = str(Seq(readout_from_adaptor).reverse_complement())
    print('readout from adaptor rc:', readout_seq)


    readout_seq_quote = readout_df['sequence'].tolist()[0]
    readout_seq_quote = readout_seq_quote.split('/')[-1].replace(' ', '')
    print('readout from quote:     ', readout_seq_quote)
    
    
    print(readout_seq == readout_seq_quote)
    

Check Stv82_ss_Alex750N
readout on adaptor: TTTGCACTGCCGTCCTTGAC
  
readout from adaptor rc: GTCAAGGACGGCAGTGCAAA
readout from quote:      GTCAAGGACGGCAGTGCAAA
True
Check Stv1_ss_Alex647N
readout on adaptor: TTTGCACTGCCGTCCTTGAC
  
readout from adaptor rc: GTCAAGGACGGCAGTGCAAA
readout from quote:      GTCAAGGACGGCAGTGCAAA
True
Check Stv79_ss_Atto565N
readout on adaptor: TTTGCACTGCCGTCCTTGAC
  
readout from adaptor rc: GTCAAGGACGGCAGTGCAAA
readout from quote:      GTCAAGGACGGCAGTGCAAA
True


In [125]:
_name = 'Common readouts'
csv_fname = os.path.join(output_sub_folder, f'{_name}.csv')
if not os.path.exists(csv_fname):
    print(f'Save {_name}')
    readout_df.to_csv(csv_fname, index=False)
else:
    print(f'Skip {_name}')

Save Common readouts


# 4. Save all probes as seperate excel sheet

In [126]:
excel_fname = os.path.join(output_main_folder, f'Table_S3_MOp_primer_and_readout_oligopropes.xlsx')

if not os.path.exists(excel_fname):
    print(f'Save {excel_fname}')
    with pd.ExcelWriter(excel_fname) as writer:  
        for _name_csv in os.listdir(output_sub_folder):
            _df = pd.read_csv(os.path.join(output_sub_folder,_name_csv))
            _name = _name_csv.split('.csv')[0]
            _df.to_excel(writer, sheet_name=_name, index=False)
        
else:
    print(f'Skip {excel_fname}')
        

Save L:\Shiwei\Figures\MOp_draft_2023_v2\Supplementary_tables\Table_3_MOp_primer_and_readout_oligopropes.xlsx
